# first test model

We try here to clasify the waves to language

In [1]:
import polars as pl
import numpy as np
import os

In [4]:
dfTrain = pl.read_csv("../data/train_clean.csv").sample(fraction=1.0, shuffle=True)
dfTrain.head()

client_id,path,language,label
i64,str,str,i64
896,"""/Swedish/train/swd_trn_sp_119/…","""Swedish""",22
132,"""/Kyrgyz/train/kyr_trn_sp_46/co…","""Kyrgyz""",16
694,"""/Breton/train/brt_trn_sp_83/co…","""Breton""",11
1577,"""/Indonesian/train/indo_trn_sp_…","""Indonesian""",27
63,"""/Greek/train/grk_trn_sp_25/com…","""Greek""",7


In [28]:
import torch
from torch.utils.data import Dataset
import torchaudio as ta
import torchaudio.transforms as T

SAMPLE_RATE = 16000
MEL_TRANSFORM = T.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=64
)

class VoiceDataset(Dataset):
    def __init__(self, df:pl.DataFrame, transform=MEL_TRANSFORM, target_sr=SAMPLE_RATE, root="."):
        self.paths = df['path']
        self.labels = df['label']
        self.transform = transform
        self.target_sr = target_sr
        self.root = root

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        path = self.root + self.paths[index]

        waveform, sample_rate = ta.load(path)
        
        if sample_rate != self.target_sr:
            waveform = ta.functional.resample(waveform,sample_rate, self.target_sr)

        if self.transform is not None:
            features = self.transform(waveform)
            features = torch.cat([features, features], dim=2)
        else:
            features = waveform

        label = self.labels[index]

        return features, label

In [35]:
from torch.utils.data import DataLoader

dataset = VoiceDataset(dfTrain, root="../data/common_voice_kpd")

def pad_collate(batch):
    features = [item[0].squeeze(0).transpose(0,1) for item in batch]  # (n_mels, T) → (T, n_mels) pour LSTM
    labels = [item[1] for item in batch]

    lengths = torch.tensor([f.shape[0] for f in features])

    padded = torch.nn.utils.rnn.pad_sequence(features, batch_first=True)

    return padded, torch.tensor(labels), lengths

loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=pad_collate
)

In [32]:
len(dfTrain)

22194

In [36]:
min = len(dataset[0][0][0][0])
for i in range(22194):
    if len(dataset[i][0][0][0]) <min:
        min = len(dataset[i][0][0][0])

print(min)

56


In [ ]:
dataset[0][0].shape

torch.Size([1, 64, 460])

In [39]:
50*65

3250

## Dataset

In [6]:
import torch
from torch.utils.data import Dataset
import torchaudio

SAMPLE_RATE = 16000

class AudioDataset(Dataset):
    def __init__(self, df, root=""):
        self.paths = df["path"]
        self.labels = df["label"]
        self.root = root

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        waveform, sr = torchaudio.load(self.root + self.paths[idx])

        if sr != SAMPLE_RATE:
            waveform = torchaudio.functional.resample(
                waveform, sr, SAMPLE_RATE
            )

        waveform = waveform.squeeze(0)  # [T]
        label = self.labels[idx]

        return waveform, label

dataset = AudioDataset(dfTrain, root="../data/common_voice_kpd")

In [15]:
dataset[260][0].shape

torch.Size([59136])

## Convolution Neural Network

here we transform sequential waves to a (3250,) vector. Where we try to find a latent space for later put them in a RNN to get 